In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(access_token)

APP_USR-6628000663056293-110612-a39a51193dac91a77f8fc723fb937f6d-233632476


## Verificando dados da conta

In [ ]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

## Pegando Todos os Itens do ML

Por padrão a API do ML limita a exibição de até 50 itens por página. Usamos o parâmetro 'limit' para ampliar esse valor a 100 itens/página. Desta forma, a lista de páginas diminui mas ainda precisamos percorrer toda ela. Para isso precisaremos do scroll_id que percorre todas as páginas uma de cada vez em sequência.

### Pegando scroll_id e os primeiros 100 itens


In [ ]:
resultados = []
url = f"https://api.mercadolibre.com/users/{seller_id}/items/search?search_type=scan"

payload = {}

params = {
    'limit': 100,  # Defina o número de resultados por página (máximo 50)
    'offset': 0,  # Defina o deslocamento (página) que você deseja recuperar
}

headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response = response.text

# print(response)

json_data = response
data = json.loads(json_data)
resultados.extend(data.get("results", []))

# Acesse o valor da chave "scroll_id"
scroll_id = data.get("scroll_id")

print(f'O valor de "scroll_id" é: {scroll_id}')


In [ ]:
# Quantidade de itens encontrados
len(resultados)

### Pegando o restante de itens em cada uma das páginas

Por padrão a API do ML limita a exibição de até 50 itens por página.

In [ ]:
url_base = f"https://api.mercadolibre.com/users/{seller_id}/items/search?search_type=scan&scroll_id={scroll_id}"

# Defina os cabeçalhos
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Quantidade de repetições
repeticoes = 365

for i in range(repeticoes):
    # Faça a solicitação GET
    response = requests.get(url_base, headers=headers)

    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Carregue o JSON da resposta
        data = response.json()

        # Acesse os itens da chave "results" e adicione-os à lista "resultados"
        resultados.extend(data.get("results", []))
        # print(resultados)
    else:
        print(f'Erro na solicitação da repetição {i + 1}: {response.status_code} - {response.text}')
        if response.status_code == 404:
            print('Acabou a lista')
            break

In [ ]:
# Registra o tempo depois da execução
end_prog = time.time()

# Calcula o tempo decorrido
elapsed_time = end_prog - start_prog

print(f"Tempo Total do processo: {elapsed_time / 60} minutos")